In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import poisson
import time
import matplotlib.pyplot as plt
import plotly.express as px


## Functions

In [2]:
I = 48  # Number of intervals
d = 5  # length of interval
x = np.random.choice(5, I)
N = sum(x)
precision = 0.9999
beta = 9  # Average service time for a patient
no_show = 0 # Fraction of scheduled patients not showing up
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4

print(N)

99


In [3]:
def calcExponentialLimit(mu):
    return int(max(mu+4*mu**0.5, 100))

In [4]:
def binomCoeff(k, i):
  return math.factorial(k) / (math.factorial(k - i) * math.factorial(i))


def binomPMF(k, i, m, add_v, no_show):
  return binomCoeff(k, m) * add_v[m][i] * (1 - no_show)**m * no_show**(k-m)

In [5]:
# Distribution to calculate service time of patients
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.
def calculate_p(beta, size, precision=0.9999):  # Poisson distribution
    k = 0
    p = []

    while sum(p) < precision:  # fill accurate values up to precision limit
        p.append(poisson.pmf(k, beta))
        k += 1

    while len(p) < size:  # fill the rest of the values with 0
        p.append(0)
    return p, k

##########
## TEST ##
##########

probS = calcExponentialLimit(beta*N)+1
print(probS)
calculate_p(beta, probS, precision)


1011


([0.00012340980408667956,
  0.0011106882367801166,
  0.004998097065510523,
  0.014994291196531574,
  0.033737155192196056,
  0.06072687934595293,
  0.09109031901892926,
  0.1171161244529091,
  0.13175564000952278,
  0.13175564000952278,
  0.11858007600857066,
  0.09702006218883041,
  0.0727650466416229,
  0.050375801521123684,
  0.03238444383500792,
  0.01943066630100472,
  0.010929749794315179,
  0.005786338126402172,
  0.0028931690632010803,
  0.0013704485036215655,
  0.0006167018266297063,
  0.0002643007828413016,
  0.00010812304752598687,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [6]:
def calculate_v(p, beta, precision, precision_limit, N, d, no_show=0):
  count = precision_limit
  limit = calcExponentialLimit(beta*N) + 1
  v = np.zeros((N+1, limit+d))
  add_v = np.zeros((N+1, limit+d))

  add_v[0][0] = 1
  for k in range(1, N+1):
    limit = calcExponentialLimit(beta*k)
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      z = 0
      while z <= count:
        add_v[k][i] += p[z] * add_v[k-1][i-z]
        z += 1
      sum_v += add_v[k][i]
      i += 1

  for k in range(N+1):
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      for m in range(k+1):
        v[k][i] += binomPMF(k, i, m, add_v, no_show)
      sum_v += v[k][i]
      i += 1

  return v, limit

##########
## TEST ##
##########

p, precision_limit = calculate_p(beta, probS, precision)
calculate_v(p, beta, precision, precision_limit, N, d, no_show)


(array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.23409804e-04, 1.11068824e-03, 4.99809707e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.52299797e-08, 2.74139635e-07, 2.46725672e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [5.46064214e-08, 2.64794969e-08, 1.21467926e-08, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 1010)

In [7]:
def calculateProbabilities(x, precision, limit, v, I, d):
  p_plus = np.zeros((I+1, limit+d+1))
  p_min = np.zeros((I+1, limit+d+1))

  # Constraint 1
  p_min[0][0] = 1

  # Constraint 2
  sum_p = 0
  i = 0
  while sum_p < precision and i <= limit:
    p_plus[0][i] = v[x[0]][i]
    sum_p += p_plus[0][i]
    i += 1

  for t in range(1, I+1):  # calculate p_min and p_plus iteratively
    # Constraint 3
    for k in range(d+1):
      # probability of amount of work = 0 just before the start of t equals the cummulative probablity of amount of work less or equal to duration of the interval just at the start of the previous interval, t-1
      p_min[t][0] += p_plus[t-1][k]
    # Constraint 4
    for i in range(1, limit+1):

      # probability of amount of work = i just before the start of interval t equals the probablity of amount of work exceeding the duration of the interval by i just at the start of the previous interval, t-1
      p_min[t][i] = p_plus[t-1][i+d]

    # Constraint 5
    if t != I:  # I or I+1
      for i in range(limit+1):
        for j in range(i+1):
          p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

  return p_min, limit

##########
## TEST ##
##########

v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)
p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)


In [8]:
def calcTardiness(p_min, limit, I):
  tardiness = 0
  # print(p_min[I])
  for k in range(limit):
    tardiness += k * p_min[I][k]  # I+1
  return tardiness


def calcIdletime(I, d, tardiness, N, no_show, beta):
  return (I * d) + tardiness - (N * (1 - no_show) * beta)  # I-1?


def calcWaitingtime(p_min, x, p, limit, I, N):
  w = np.zeros((I+1, N+1, limit+1))
  waitingtime = 0

  for t in range(0, I):
    if x[t] > 0:
      for k in range(limit):
        w[t][0][k] = p_min[t][k]
    if x[t] > 1:
      for i in range(1, x[t]+1):
        for k in range(limit+1):
          for j in range(k+1):
            w[t][i][k] += w[t][i-1][j] * p[k-j]

  for t in range(0, I):
    for i in range(0, x[t]):
      for k in range(limit+1):
        waitingtime += w[t][i][k] * k

  waitingtime /= N
  return waitingtime

##########
## TEST ##
##########

calcWaitingtime(p_min, x, p, limit, I, N)


327.3967060245299

In [9]:
def calcFracExcess(p_min, I):
  fracExcess = 0
  t = I+1
  for j in range(1, len(p_min[t])):
    fracExcess += p_min[t][j]
  fracExcess *= 100
  return fracExcess


##########
## TEST ##
##########

# calcFracExcess(p_min, I)


<div>
<img src="https://github.com/witusj/obp/blob/master/images/fitness2.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/fitness.png?raw=true" width="450">
</div>

In [10]:
def calcFitness(x, beta, precision, limit, v, N, no_show, I, d, eind, alpha_W, alpha_I, alpha_T):
  tic = time.perf_counter()
  p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)
  toc = time.perf_counter()
  probT = toc-tic

  # Tardiness calcs
  tic = time.perf_counter()
  tardiness = calcTardiness(p_min, limit, I)
  toc = time.perf_counter()
  tardT = toc-tic

  # Idle time calcs new array of given shape and type, filled with zeros.
  tic = time.perf_counter()
  idletime = calcIdletime(I, d, tardiness, N, no_show, beta)
  toc = time.perf_counter()
  idletimeT = toc-tic

  # Waiting time calcs
  tic = time.perf_counter()
  waitingtime = calcWaitingtime(p_min, x, p, limit, I, N)
  toc = time.perf_counter()
  waitingtimeT = toc-tic

  objVal = alpha_W*waitingtime + alpha_I*idletime + alpha_T*tardiness

  print(f"Schedule: {x},\nN: {sum(x)},\nObjective value: {objVal},\nProb calculation time: {probT:.6f} sec,\nWaiting time (timer): {waitingtime} ({waitingtimeT:.6f} sec),\nIdle time (timer): {idletime} ({idletimeT:.6f} sec),\nTardiness (timer): {tardiness} ({tardT:.6f} sec)\n")

  # Collect into a dictionary
  results = {'p_min': p_min, 'waitingTime': waitingtime,
            'idleTime': idletime, 'tardiness': tardiness, 'objVal': objVal}

  if eind == 1:
    fracExcess = calcFracExcess(p_min, I)
    results['fracExcess'] = fracExcess

  return results

##########
## TEST ##
##########


p, precision_limit = calculate_p(beta, probS, precision)
v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)



results = calcFitness(x, beta, precision, limit, v, N, no_show,
                  I, d, eind, alpha_W, alpha_I, alpha_T)
results


Schedule: [2 4 4 1 2 2 0 3 2 3 0 1 0 4 3 3 4 0 3 3 4 4 0 2 3 0 0 1 3 3 3 2 4 3 1 1 3
 0 2 4 0 2 0 2 3 2 1 2],
N: 99,
Objective value: 388.72556380882867,
Prob calculation time: 21.151039 sec,
Waiting time (timer): 327.3967060245299 (44.564778 sec),
Idle time (timer): -4.388531001638853 (0.000002 sec),
Tardiness (timer): 646.6114689983611 (0.000356 sec)



{'p_min': array([[1.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [3.23993451e-004, 7.19452059e-004, 1.85001958e-003, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [6.12205184e-014, 3.50213518e-013, 1.97428371e-012, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        ...,
        [7.90326651e-147, 3.50057170e-146, 1.71533340e-145, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [1.75551179e-146, 6.20096972e-146, 2.66437582e-145, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [2.02374319e-149, 7.89334473e-149, 3.50524582e-148, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000]]),
 'waitingTime': 327.3967060245299,
 'idleTime': -4.388531001638853,
 'tardiness': 646.6114689983611,
 'objVal': 388.72556380882867}

In [11]:
def createDist(arr, reverse=False):
    minX, maxX = arr.min(), arr.max()
    minY, maxY = 0, 1
    if(reverse):
        minY, maxY = maxY, minY
    # Scale to values between 0, 1 and make total sum equal to 1
    arrScaled = np.interp(arr, (minX, maxX), (minY, maxY))
    arrNorm = [x / arrScaled.sum() for x in arrScaled]
    return(arrNorm)
    

##########
## TEST ##
##########

arr = np.random.choice(10, 5)
dist1 = createDist(arr)
dist2 = createDist(arr, reverse=True)
arr, dist1, sum(dist1), dist2, sum(dist2)


(array([9, 8, 4, 8, 3]),
 [0.35294117647058826,
  0.29411764705882354,
  0.05882352941176471,
  0.29411764705882354,
  0.0],
 1.0,
 [0.0,
  0.07692307692307694,
  0.3846153846153846,
  0.07692307692307694,
  0.46153846153846145],
 0.9999999999999999)

<div>
<img src="https://github.com/witusj/obp/blob/master/images/crossover.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/crossover2.png?raw=true" width="450">
</div>

In [12]:
## OLD
def crossover(parent1, parent2, nSwaps):
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    I = len(child1)
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    # norm1 = sum(child1)
    # p1 = child1 / norm1
    p1 = createDist(child1)
    
    # The number of swaps should not exceed the maximum number of patients in any interval. Otherwise it might happen that the number of patients that are transfered exceeds availability, resulting in a negative amount.
    max1 = int(max(child1))
    max2 = int(max(child2))
    nSwaps = min(nSwaps, max1, max2)
    
    # Swap patients parent1 to parent 2
    i = np.random.choice(I, nSwaps, p=p1)
    for i in i:
        child1[i] += -1
        child2[i] += 1
    
    # Swap patients parent2 to parent 1
    # norm2 = sum(child2)
    # p2 = child2 / norm2
    p2 = createDist(child2)
    
    j = np.random.choice(I, nSwaps, p=p2)
    for j in j:
        child1[j] += 1
        child2[j] += -1
    return(child1, child2)

##########
## TEST ##
##########


parent1 = np.random.choice(5, I)
parent2 = np.random.choice(5, I)

print(parent1, parent2)
nSwaps = 3
children = crossover(parent1, parent2, nSwaps)
children


[0 3 1 2 2 2 1 1 0 1 1 0 4 0 2 1 0 4 3 4 2 2 4 2 2 1 0 2 2 1 3 1 1 3 2 4 0
 2 2 2 3 2 4 3 1 2 4 2] [3 4 0 2 3 1 2 3 4 2 4 1 1 2 4 4 2 4 3 1 2 0 3 4 3 3 1 2 3 1 0 4 2 1 2 2 3
 4 1 1 0 3 4 0 2 4 3 0]


(array([0, 3, 1, 2, 2, 2, 1, 1, 0, 2, 1, 0, 4, 0, 2, 1, 1, 4, 3, 4, 2, 2,
        4, 2, 2, 1, 0, 2, 2, 1, 3, 1, 1, 3, 1, 4, 0, 2, 2, 2, 3, 2, 3, 3,
        1, 2, 4, 2]),
 array([3, 4, 0, 2, 3, 1, 2, 3, 4, 1, 4, 1, 1, 2, 4, 4, 1, 4, 3, 1, 2, 0,
        3, 4, 3, 3, 1, 2, 3, 1, 0, 4, 2, 1, 3, 2, 3, 4, 1, 1, 0, 3, 5, 0,
        2, 4, 3, 0]))

In [13]:
def crossover(parent1, parent2, nSwaps):
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    I = len(child1)
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    p1 = createDist(child1)
    
    # Swap patients parent1 to parent 2
    n1 = nSwaps
    while(n1 > 0):
        i = np.random.choice(I, 1, p=p1)
        child1[i] += -1
        child2[i] += 1
        if(child1[i] == 0):
            p1 = createDist(child1) # If the number of patients in interval i reaches zero, this interval should not be selected again.
        n1 -= 1
    
    # Swap patients parent2 to parent 1
    p2 = createDist(child2)
    
    n2 = nSwaps
    while(n2 > 0):
        i = np.random.choice(I, 1, p=p2)
        child2[i] += -1
        child1[i] += 1
        if(child2[i] == 0):
            p2 = createDist(child2)
        n2 -= 1
        
    return(child1, child2)

##########
## TEST ##
##########

testI = 10
parent1 = np.random.choice(5, testI)
parent2 = np.random.choice(5, testI)

print(parent1, parent2)
nSwaps = 3
children = crossover(parent1, parent2, nSwaps)
children


[3 3 0 4 0 3 2 1 1 2] [3 0 0 3 3 3 0 1 3 1]


(array([2, 3, 0, 4, 0, 3, 2, 2, 1, 2]), array([4, 0, 0, 3, 3, 3, 0, 0, 3, 1]))


<div>
    <img src="https://github.com/witusj/obp/blob/master/images/mutation.png?raw=true" width="450">
    <img src="https://github.com/witusj/obp/blob/master/images/mutation1.png?raw=true" width="450">
</div>

In [14]:
def mutate(child, rate):
    if np.random.rand() < rate:
        x = len(child)
        i, j = np.random.choice(x, 2, replace=False).astype(int)
        a, b = child[i], child[j]
        child[i], child[j] = b, a
    return(child)

##########
## TEST ##
##########

child = children[0].copy()
print(child)
child = mutate(child, 1)
print(child)


[2 3 0 4 0 3 2 2 1 2]
[2 3 0 1 0 3 2 2 4 2]


## Classes

#### Suggesties: [get / set methods](https://www.geeksforgeeks.org/getter-and-setter-in-python/) voor parameters / met tests / nuttig voor optimaliseren hyper parameters, bv mbv bayesian stats

Create a data object to store and generate initial data.

In [15]:
class Data:
    def __init__(self, I=10, N=12, d=5, popSize=5, beta=9, precision=0.9999, no_show=0, eind=0, alpha_I=0.2, alpha_T=0.3, alpha_W=0.5, nSwaps=4, mutationRate=0.1):
        self.I = I
        self.N = N
        self.d = d
        self.popSize = popSize
        self.beta = beta
        self.precision = precision
        self.no_show = no_show
        self.probSize = calcExponentialLimit(self.beta*self.N)+1
        self.p, self.precision_limit = self.__calculate_p()
        self.v, self.limit = self.__calculate_v()
        self.eind = eind
        self.alpha_I, self.alpha_T, self.alpha_W = alpha_I, alpha_T, alpha_W
        self.nSwaps, self.mutationRate = nSwaps, mutationRate
    
    def get_initial_data(self): return np.array((self.I, self.N, self.d, self.beta, self.popSize, self.eind, self.alpha_I, self.alpha_T, self.alpha_W, self.nSwaps, self.mutationRate),
                                                dtype={'names': ('I', 'N', 'd', 'beta', 'popSize', 'eind', 'alpha_I', 'alpha_T', 'alpha_W', 'nSwaps', 'mutationRate'),
                                                'formats': ('i4', 'i4', 'i4', 'i4', 'i4', 'i4', 'f8', 'f8', 'f8', 'i4', 'f8')})

    def get_prob_data(self): return np.array((self.precision, self.no_show, self.probSize, self.precision_limit, self.limit),
                                                dtype={'names': ('precision', 'no_show', 'probSize', 'precision_limit', 'limit'),
                                                'formats': ('f8', 'f8', 'i4', 'f8', 'i4')})

    def calculate_p(self):  return calculate_p(self.beta, self.probSize, self.precision)
    __calculate_p = calculate_p
    
    def calculate_v(self): return calculate_v(self.p, self.beta, self.precision, self.precision_limit, self.N, self.d, self.no_show)
    __calculate_v = calculate_v
    

##########
## TEST ##
##########

newDataObject = Data(I, N)
initData = newDataObject.get_initial_data()
initData[['N', 'popSize']]


array((99, 5),
      dtype={'names': ['N', 'popSize'], 'formats': ['<i4', '<i4'], 'offsets': [4, 16], 'itemsize': 60})

<img src="https://github.com/witusj/obp/blob/master/images/population.png?raw=true" width="600">

Create a random individual. An individual is in this case one schedule.

In [16]:
class Schedule:
    def __init__(self, dataObject):
        self.I = dataObject.get_initial_data()['I']  # Number of intervals
        self.N = dataObject.get_initial_data()['N']  # Number of patients
        self.x = np.zeros(self.I).astype(int)# Empty schedule

    def create_random_schedule(self):
        for i in range(self.N):
            i = np.random.choice(self.I, 1)  # Interval ID
            self.x[i] = self.x[i] + 1
    def get_schedule(self): return(self.x)
    def print_schedule(self): print(self.x)


##########
## TEST ##
##########

dataObject = Data(I=I, N=N, d=d)
scheduleObject = Schedule(dataObject)
scheduleObject.print_schedule()
scheduleObject.create_random_schedule()
schedule = scheduleObject.get_schedule()
schedule

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]


array([2, 3, 4, 0, 3, 3, 3, 3, 3, 2, 6, 1, 4, 1, 2, 4, 1, 1, 2, 3, 3, 4,
       1, 1, 1, 0, 1, 1, 4, 2, 1, 0, 3, 0, 1, 4, 0, 4, 2, 3, 1, 0, 1, 2,
       1, 2, 3, 2])

Create population of individuals

In [17]:
class Population:
    global bestRank
    bestRank = []
    def __init__(self, dataObject, individuals=None):
        self.dataObject = dataObject
        self.params = self.dataObject.get_initial_data()
        self.prob_params = self.dataObject.get_prob_data()
        self.p, self.v = self.dataObject.p, self.dataObject.v
        self.individuals = individuals
        if self.individuals == None: self.__create_new_individuals() 
        self.fitnessScores = []

    def create_new_individuals(self):
        self.individuals = []
        for i in range(self.params['popSize']):
            x = Schedule(self.dataObject)
            x.create_random_schedule()
            x = x.get_schedule()
            self.individuals.append(x)
            
    __create_new_individuals = create_new_individuals
    
    def get_individuals(self): return(self.individuals)
    
    def calc_fitness_scores(self):
            beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[['beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
            precision, limit, no_show = self.prob_params[['precision', 'limit', 'no_show']].tolist()
            self.fitnessScores = np.array([calcFitness(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T)['objVal'] for x in self.individuals])
            
    def apply_crossover(self):
        newIndividuals = []
        nSwaps = self.params['nSwaps']*1 ## Times 1 because otherwise refers to memorypoint
        mutationRate = self.params['mutationRate']
        self.population_dist = createDist(self.fitnessScores, reverse=True)
        
        for i in range(self.params['popSize']):
            print(f'Crossover {i}')
            r1, r2 = np.random.choice(self.params['popSize'], 2, replace=False, p=self.population_dist)
            parent1, parent2 = self.individuals[r1], self.individuals[r2]
            child1, child2 = crossover(parent1, parent2, nSwaps)
            newIndividuals.append(mutate(child1, mutationRate))
            newIndividuals.append(mutate(child2, mutationRate))
            
        beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[[
            'beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
        precision, limit, no_show = self.prob_params[[
            'precision', 'limit', 'no_show']].tolist()
        self.fitnessScores = np.array([calcFitness(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show,
                                            I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T)['objVal'] for x in newIndividuals])
        tempList = list(range(self.params['popSize']*2))
        zipped = list(zip(*sorted(zip(self.fitnessScores, tempList))))
        self.fitnessScores = np.array(list(zipped[0][:self.params['popSize']]))
        tempList = zipped[1][:self.params['popSize']]
        self.individuals = list(map(newIndividuals.__getitem__, tempList))
        bestRank.append([self.fitnessScores[0], self.individuals[0]])
        print(self.fitnessScores, '\n', self.individuals)


## Tests

In [18]:
##########
## TEST ##
##########

bestRank = []
dataObject = Data()
populationObject1 = Population(dataObject)
populationObject1.fitnessScores
populationObject1.calc_fitness_scores()
print(populationObject1.fitnessScores)


Schedule: [1 2 2 0 1 1 1 1 1 2],
N: 12,
Objective value: 30.941760276954625,
Prob calculation time: 0.094935 sec,
Waiting time (timer): 27.057942716062325 (0.072027 sec),
Idle time (timer): 0.025577837846924467 (0.000002 sec),
Tardiness (timer): 58.02557783784693 (0.000064 sec)

Schedule: [0 0 0 3 1 1 1 2 2 2],
N: 12,
Objective value: 42.10357626840413,
Prob calculation time: 0.092260 sec,
Waiting time (timer): 34.477340824191124 (0.106291 sec),
Idle time (timer): 14.929811712617138 (0.000002 sec),
Tardiness (timer): 72.92981171261714 (0.000064 sec)

Schedule: [2 1 2 1 0 1 0 1 3 1],
N: 12,
Objective value: 31.07004290828669,
Prob calculation time: 0.097005 sec,
Waiting time (timer): 27.39797420147832 (0.090258 sec),
Idle time (timer): -0.05788838490494186 (0.000003 sec),
Tardiness (timer): 57.942111615095065 (0.000075 sec)

Schedule: [1 1 1 1 1 1 2 1 0 3],
N: 12,
Objective value: 29.499377914782507,
Prob calculation time: 0.086160 sec,
Waiting time (timer): 24.15605799255908 (0.052186 

In [19]:
individuals2 = populationObject1.individuals.copy()
populationObject2 = Population(dataObject, individuals=individuals2)
print(populationObject1.individuals, '\n', populationObject2.individuals)

[array([1, 2, 2, 0, 1, 1, 1, 1, 1, 2]), array([0, 0, 0, 3, 1, 1, 1, 2, 2, 2]), array([2, 1, 2, 1, 0, 1, 0, 1, 3, 1]), array([1, 1, 1, 1, 1, 1, 2, 1, 0, 3]), array([2, 2, 2, 0, 0, 0, 1, 0, 1, 4])] 
 [array([1, 2, 2, 0, 1, 1, 1, 1, 1, 2]), array([0, 0, 0, 3, 1, 1, 1, 2, 2, 2]), array([2, 1, 2, 1, 0, 1, 0, 1, 3, 1]), array([1, 1, 1, 1, 1, 1, 2, 1, 0, 3]), array([2, 2, 2, 0, 0, 0, 1, 0, 1, 4])]


In [20]:
populationObject2.calc_fitness_scores()
populationObject2.apply_crossover()
populationObject2.individuals


Schedule: [1 2 2 0 1 1 1 1 1 2],
N: 12,
Objective value: 30.941760276954625,
Prob calculation time: 0.089060 sec,
Waiting time (timer): 27.057942716062325 (0.063737 sec),
Idle time (timer): 0.025577837846924467 (0.000002 sec),
Tardiness (timer): 58.02557783784693 (0.000055 sec)

Schedule: [0 0 0 3 1 1 1 2 2 2],
N: 12,
Objective value: 42.10357626840413,
Prob calculation time: 0.079308 sec,
Waiting time (timer): 34.477340824191124 (0.095042 sec),
Idle time (timer): 14.929811712617138 (0.000002 sec),
Tardiness (timer): 72.92981171261714 (0.000053 sec)

Schedule: [2 1 2 1 0 1 0 1 3 1],
N: 12,
Objective value: 31.07004290828669,
Prob calculation time: 0.078138 sec,
Waiting time (timer): 27.39797420147832 (0.080577 sec),
Idle time (timer): -0.05788838490494186 (0.000002 sec),
Tardiness (timer): 57.942111615095065 (0.000052 sec)

Schedule: [1 1 1 1 1 1 2 1 0 3],
N: 12,
Objective value: 29.499377914782507,
Prob calculation time: 0.088728 sec,
Waiting time (timer): 24.15605799255908 (0.052461 

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 3]),
 array([1, 1, 3, 0, 0, 0, 2, 1, 1, 3]),
 array([2, 1, 0, 1, 1, 1, 1, 2, 1, 2]),
 array([1, 2, 1, 1, 0, 1, 1, 1, 2, 2]),
 array([2, 2, 0, 1, 1, 1, 1, 0, 0, 4])]

In [21]:
# Initial population
bestRank = []
dataObject = Data()
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
individuals = populationZero.individuals.copy()
generations = 3

# Genetic algorithm loop
for i in range(generations):
    print(f'Generation: {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.calc_fitness_scores()
    populationObject.apply_crossover()
    individuals = populationObject.individuals


Schedule: [1 0 1 0 4 0 3 1 1 1],
N: 12,
Objective value: 32.63744212896203,
Prob calculation time: 0.086976 sec,
Waiting time (timer): 27.31315378229277 (0.081386 sec),
Idle time (timer): 3.1617304756312876 (0.000002 sec),
Tardiness (timer): 61.16173047563129 (0.000058 sec)

Schedule: [3 0 0 0 3 0 0 1 3 2],
N: 12,
Objective value: 29.5569264741992,
Prob calculation time: 0.088968 sec,
Waiting time (timer): 24.196826513254123 (0.124088 sec),
Idle time (timer): 0.11702643514428246 (0.000002 sec),
Tardiness (timer): 58.117026435144275 (0.000054 sec)

Schedule: [0 1 1 1 3 0 1 3 1 1],
N: 12,
Objective value: 34.287096603219084,
Prob calculation time: 0.094312 sec,
Waiting time (timer): 28.7364138167672 (0.072045 sec),
Idle time (timer): 5.037779389670959 (0.000002 sec),
Tardiness (timer): 63.037779389670966 (0.000054 sec)

Schedule: [1 3 2 1 0 0 1 1 1 2],
N: 12,
Objective value: 31.983060379312604,
Prob calculation time: 0.089639 sec,
Waiting time (timer): 29.14056486095552 (0.093876 sec),


In [22]:

bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF


,Score,Schedule
0,29.693760,"[1, 2, 1, 0, 2, 0, 1, 1, 1, 3]"
1,28.874364,"[1, 2, 0, 1, 1, 0, 1, 2, 1, 3]"
2,28.873702,"[1, 2, 0, 1, 1, 0, 2, 1, 0, 4]"


In [23]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 10
generations = 5

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [1 0 0 1 0 1 1 0 0 1 0 1 0 1 0 0 0 1 0 1 0 1 0 0 2 0 1 1 0 1 0 1 0 1 1 0 1
 0 2 1 0 0 0 0 0 0 0 2],
N: 24,
Objective value: 16.415721321614296,
Prob calculation time: 1.424838 sec,
Waiting time (timer): 7.082948792929081 (0.212925 sec),
Idle time (timer): 38.6375696740711 (0.000002 sec),
Tardiness (timer): 14.637569674071102 (0.000094 sec)

Schedule: [1 1 1 0 2 0 0 1 0 1 0 0 0 0 0 1 0 1 1 0 1 1 0 0 1 1 0 0 0 0 0 1 1 0 1 0 0
 1 1 0 0 2 0 1 1 0 0 1],
N: 24,
Objective value: 14.243650153810972,
Prob calculation time: 1.417279 sec,
Waiting time (timer): 5.928995026394649 (0.145796 sec),
Idle time (timer): 35.786753572088514 (0.000002 sec),
Tardiness (timer): 11.786753572088523 (0.000094 sec)

Schedule: [0 1 0 0 1 0 0 0 1 0 0 0 3 1 0 1 0 1 0 0 0 0 0 0 2 2 0 0 0 1 1 0 1 1 0 0 1
 2 1 0 1 0 1 0 0 0 1 0],
N: 24,
Objective value: 20.68390764297081,
Prob calculation time: 1.429524 sec,
Waiting time (timer): 15.01040509497446 (0.317125 sec),
Idle time (timer): 40.466242674968385 (0.00000

In [24]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score', hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='LimeGreen')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [25]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 10

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind,
                    alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop
score_list = []
for i in range(generations):
    print(f'Generation {i}')
    tic = time.perf_counter()
    populationObject = Population(dataObject, individuals=individuals)
    toc = time.perf_counter()
    objTime = toc - tic
    populationObject.fitnessScores = fitnessScores
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    tic = time.perf_counter()
    populationObject.apply_crossover()
    toc = time.perf_counter()
    crossTime = toc - tic
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()
    print(f'Population building time: : {objTime}\nCrossover time: {crossTime}\n')


Schedule: [0 1 0 2 0 0 1 0 0 0 0 0 0 2 0 0 2 0 0 0 0 0 1 1 0 1 0 2 0 0 1 1 0 1 2 0 0
 2 0 0 0 0 1 0 1 0 1 1],
N: 24,
Objective value: 18.69780353224024,
Prob calculation time: 1.435567 sec,
Waiting time (timer): 8.890768752437154 (0.411646 sec),
Idle time (timer): 41.23582671877563 (0.000002 sec),
Tardiness (timer): 17.235826718775627 (0.000153 sec)

Schedule: [0 4 0 0 1 0 3 1 0 0 1 1 1 0 1 1 0 0 0 0 0 2 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 2 0 0 1 2 1 0 0 0 0 0],
N: 24,
Objective value: 16.631872848137682,
Prob calculation time: 1.410863 sec,
Waiting time (timer): 25.27332443180012 (0.452207 sec),
Idle time (timer): 26.870905125696055 (0.000002 sec),
Tardiness (timer): 2.8709051256960554 (0.000098 sec)

Schedule: [1 0 0 0 1 1 0 1 0 1 0 0 0 0 1 2 1 0 0 1 0 2 0 1 1 0 0 1 0 1 0 0 0 0 0 1 2
 0 0 1 0 0 1 0 0 2 1 0],
N: 24,
Objective value: 16.738249795559653,
Prob calculation time: 1.412576 sec,
Waiting time (timer): 9.20309840473103 (0.275193 sec),
Idle time (timer): 37.761684056112074 (0.00000

In [26]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='RoyalBlue')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [27]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 20

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind,
                    alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop
score_list = []
for i in range(generations):
    print(f'Generation {i}')
    tic = time.perf_counter()
    populationObject = Population(dataObject, individuals=individuals)
    toc = time.perf_counter()
    objTime = toc - tic
    populationObject.fitnessScores = fitnessScores
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    tic = time.perf_counter()
    populationObject.apply_crossover()
    toc = time.perf_counter()
    crossTime = toc - tic
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()
    print(f'Population building time: : {objTime}\nCrossover time: {crossTime}\n')


Schedule: [0 1 0 1 3 1 2 0 0 0 0 0 0 1 0 0 1 0 0 2 2 1 0 1 1 0 0 0 0 0 0 0 2 0 0 0 0
 1 0 0 1 0 0 1 0 0 1 1],
N: 24,
Objective value: 15.579636993819102,
Prob calculation time: 1.357583 sec,
Waiting time (timer): 14.450650874933944 (0.365212 sec),
Idle time (timer): 32.332294406409204 (0.000002 sec),
Tardiness (timer): 8.332294406409208 (0.000091 sec)

Schedule: [0 0 1 0 0 0 0 0 0 2 1 1 0 2 0 0 0 1 1 0 0 1 0 2 1 0 0 1 0 1 2 1 1 1 2 0 0
 0 0 0 0 0 0 1 0 1 0 0],
N: 24,
Objective value: 21.819047547703,
Prob calculation time: 1.380091 sec,
Waiting time (timer): 21.877833351013496 (0.331554 sec),
Idle time (timer): 37.779857012162665 (0.000002 sec),
Tardiness (timer): 13.77985701216267 (0.000099 sec)

Schedule: [0 1 0 0 0 0 2 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 1 0 1 1 0 0 1 0 1 0 1 0 2 0 2
 0 1 1 1 2 1 1 0 0 0 0],
N: 24,
Objective value: 28.324021216147635,
Prob calculation time: 1.335089 sec,
Waiting time (timer): 12.343188135952529 (0.264931 sec),
Idle time (timer): 54.97790993627768 (0.00000

In [28]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='Tomato')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [29]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 10
generations = 5
nSwaps = 6

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize, nSwaps=nSwaps)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [0 0 0 1 2 0 1 0 0 0 2 2 0 2 2 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 1 0 1 0 0 1 0
 0 0 2 1 0 0 1 0 1 0 0],
N: 24,
Objective value: 13.230082722346221,
Prob calculation time: 1.412142 sec,
Waiting time (timer): 14.54079126106874 (0.414096 sec),
Idle time (timer): 28.356277029864543 (0.000002 sec),
Tardiness (timer): 4.356277029864539 (0.000099 sec)

Schedule: [0 1 0 1 0 1 1 0 0 2 0 2 0 1 0 1 0 0 1 3 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0
 0 2 0 0 1 0 1 1 0 0 1],
N: 24,
Objective value: 13.376920121031398,
Prob calculation time: 1.457032 sec,
Waiting time (timer): 10.653811043586817 (0.312603 sec),
Idle time (timer): 31.192326172661126 (0.000002 sec),
Tardiness (timer): 7.1923261726611125 (0.000127 sec)

Schedule: [0 0 0 2 1 1 0 0 0 0 1 0 0 0 0 1 0 0 2 0 0 1 1 0 0 0 0 0 0 1 1 0 1 1 1 1 0
 0 3 1 2 0 0 0 1 1 0 0],
N: 24,
Objective value: 29.587188541445894,
Prob calculation time: 1.409053 sec,
Waiting time (timer): 14.361928199662557 (0.406294 sec),
Idle time (timer): 55.73736210263479 (0.00

In [30]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='DarkOrange')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [31]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 5
nSwaps = 6

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize, nSwaps=nSwaps)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 2 0 2 0 2 2 0 0 0 0 3 0 0 0 0 1 1 1 0 1
 0 1 0 0 2 0 0 1 0 0 2],
N: 24,
Objective value: 33.93756558193687,
Prob calculation time: 1.390211 sec,
Waiting time (timer): 21.493420371597974 (0.508692 sec),
Idle time (timer): 58.23366238882949 (0.000002 sec),
Tardiness (timer): 34.233662388829465 (0.000095 sec)

Schedule: [1 1 2 3 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1
 0 1 0 0 2 2 0 1 1 0 0],
N: 24,
Objective value: 23.959793079279184,
Prob calculation time: 1.366183 sec,
Waiting time (timer): 15.064060548795688 (0.314145 sec),
Idle time (timer): 45.89028143293484 (0.000002 sec),
Tardiness (timer): 21.89028143293485 (0.000099 sec)

Schedule: [0 0 2 0 0 0 1 1 0 1 0 0 2 0 1 0 2 0 1 0 0 1 0 1 1 0 0 0 1 1 2 1 0 1 0 0 0
 1 1 0 0 0 0 0 1 0 1 0],
N: 24,
Objective value: 11.041065259112921,
Prob calculation time: 1.444595 sec,
Waiting time (timer): 10.668430720416623 (0.271929 sec),
Idle time (timer): 27.289488284910448 (0.000

In [32]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='DarkCyan')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")
